In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/Users/juanhernandez/Development/NWU-Pacific/NWU-Pacific/')

In [2]:
from processing import semantic_models, tokenize

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/juanhernandez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## open dataframe, get text

In [3]:
df = pd.read_csv('/Users/juanhernandez/Downloads/consumer_complaints.csv')

In [4]:
document = df['Consumer complaint narrative'][1]

In [5]:
df = df[df['Consumer complaint narrative'].isnull()==False]

## tokenize and save text

In [6]:
df['tokens'] = df['Consumer complaint narrative'].apply(tokenize.tokenize)

In [7]:
df[['Complaint ID', 'tokens']].to_hdf('/Users/juanhernandez/Documents/NWU/data/texts.hdf', 'texts')



/Users/juanhernandez/envs/pacific/lib/python2.7/site-packages/pandas-0.23.4-py2.7-macosx-10.13-x86_64.egg/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['tokens']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [8]:
texts = list(df['tokens'].values)

## make corpus and dictionary

In [9]:
dictionary = semantic_models.make_dictionary(texts, '/Users/juanhernandez/Documents/NWU/data/output/')
corpus = semantic_models.make_corpus(texts, dictionary, '/Users/juanhernandez/Documents/NWU/data/output/')

# train lda model

In [10]:
param_file = '/Users/juanhernandez/Development/NWU-Pacific/NWU-Pacific/processing/configurations/lda.json'

In [11]:
import json
with open(param_file, 'r') as config:
    parameters = json.load(config)

In [12]:
parameters['model_name'] = '_'.join([parameters['model_type'],
                                         '20',
                                         'topics'])

In [13]:
corpus_training, corpus_evaluation = semantic_models.split(corpus, parameters['training_frac'])

In [14]:
k = 20
if parameters['model_args']['alpha'] == 'k/50':
    parameters['model_args']['alpha'] = float(k)/50

In [15]:
model = semantic_models.train(parameters=parameters,
                  corpus_training=corpus_training,
                  dictionary=dictionary,
                  k=k)

In [16]:
evaluation = semantic_models.evaluate(parameters=parameters,
                          corpus_evaluation=corpus_evaluation,
                          model=model)

In [17]:
evaluation

-6.3192368634259894

## exract topics

In [ ]:
# Extract topics

from gensim.corpora import Dictionary
def get_topics(dictionary, text):
    bow = Dictionary.doc2bow(dictionary,text)
    topics = list(model.inference([bow])[0][0])
    return topics

In [ ]:
df['topics'] = df['tokens'].apply(lambda x: get_topics(dictionary, x))

In [ ]:
df[['Complaint ID', 'tokens', 'topics']].to_hdf('/Users/juanhernandez/Documents/NWU/data/topics.hdf', 'topics')

## visualize topics

In [18]:
import pyLDAvis
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()

import pandas as pd
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/Users/juanhernandez/Development/NWU-Pacific/NWU-Pacific/')
from processing import semantic_models, tokenize

import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

/Users/juanhernandez/envs/pacific/lib/python2.7/site-packages/pyLDAvis-2.1.2-py2.7.egg/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
17     11.108746        1       1  0.282261  0.209539
5       8.112953        1       2 -0.117503 -0.060246
3       7.506178        1       3 -0.183616  0.077048
15      7.439878        1       4  0.139510 -0.171710
0       6.784831        1       5  0.018833 -0.195489
1       6.045415        1       6 -0.116794  0.139206
9       5.897676        1       7 -0.127626  0.010313
10      5.565151        1       8 -0.087428 -0.135603
13      5.056764        1       9  0.044554 -0.215456
2       4.380048        1      10 -0.128770  0.034131
16      4.237746        1      11  0.092078 -0.147107
14      4.020639        1      12 -0.180177  0.114128
19      3.577734        1      13  0.138853  0.189410
8       3.507886        1      14 -0.039837 -0.020078
18      3.433755        1      15 -0.110058  0.060935
4       3.283082        1      16  0.090972 -0.117986
6       3.103058        1      17 -0.072393  0.062327
7       2.925937        1      18  0.082025  0.039114
12      2.335445        1      19  0.266893  0.077231
11      1.677077        1      20  0.008222  0.050294, topic_info=       Category          Freq        Term         Total  loglift  logprob
term                                                                     
118630  Default  2.758989e+06        xxxx  2.758989e+06  30.0000  30.0000
40201   Default  5.025670e+05     account  5.025670e+05  29.0000  29.0000
55255   Default  3.578750e+05    xxxxxxxx  3.578750e+05  28.0000  28.0000
37316   Default  4.966950e+05      credit  4.966950e+05  27.0000  27.0000
10565   Default  3.371860e+05     payment  3.371860e+05  26.0000  26.0000
41479   Default  3.706140e+05      report  3.706140e+05  25.0000  25.0000
106990  Default  2.707750e+05        loan  2.707750e+05  24.0000  24.0000
11061   Default  1.981030e+05        bank  1.981030e+05  23.0000  23.0000
63320   Default  1.824790e+05        debt  1.824790e+05  22.0000  22.0000
89632   Default  1.599740e+05        card  1.599740e+05  21.0000  21.0000
102801  Default  1.995670e+05        call  1.995670e+05  20.0000  20.0000
9642    Default  1.302130e+05       charg  1.302130e+05  19.0000  19.0000
17688   Default  1.806290e+05        send  1.806290e+05  18.0000  18.0000
73527   Default  1.307950e+05     collect  1.307950e+05  17.0000  17.0000
75044   Default  1.387090e+05      letter  1.387090e+05  16.0000  16.0000
102256  Default  1.192130e+05       check  1.192130e+05  15.0000  15.0000
75681   Default  1.223630e+05     mortgag  1.223630e+05  14.0000  14.0000
22449   Default  2.053000e+05        tell  2.053000e+05  13.0000  13.0000
72024   Default  2.358240e+05      inform  2.358240e+05  12.0000  12.0000
33568   Default  1.630820e+05       month  1.630820e+05  11.0000  11.0000
112745  Default  2.107890e+05      receiv  2.107890e+05  10.0000  10.0000
100162  Default  1.349310e+05     request  1.349310e+05   9.0000   9.0000
65178   Default  6.582600e+04     inquiri  6.582600e+04   8.0000   8.0000
17886   Default  7.845900e+04        issu  7.845900e+04   7.0000   7.0000
65970   Default  8.340100e+04       close  8.340100e+04   6.0000   6.0000
84166   Default  1.430920e+05         pay  1.430920e+05   5.0000   5.0000
84190   Default  7.860700e+04        late  7.860700e+04   4.0000   4.0000
51018   Default  9.762300e+04       remov  9.762300e+04   3.0000   3.0000
18242   Default  1.113520e+05      disput  1.113520e+05   2.0000   2.0000
56321   Default  9.226800e+04       phone  9.226800e+04   1.0000   1.0000
...         ...           ...         ...           ...      ...      ...
92480   Topic20  2.169764e+03    hurrican  2.171135e+03   4.0875  -5.3238
32599   Topic20  1.866974e+03      disast  1.868345e+03   4.0874  -5.4741
31025   Topic20  1.704401e+03   exhusband  1.705772e+03   4.0873  -5.5652
7550    Topic20  1.591989e+03        roof  1.593360e+03   4.0873  -5.6334
31784   Topic20  1.562910e+03  

In [21]:
with open('/Users/juanhernandez/Documents/NWU/data/models/lda_20_model.pkl', 'w+') as f:
    pickle.dump(model,f)